# Installs and imports

In [1]:

import os
import json
import ijson
import pandas as pd
import os
import json
from transformers import AutoTokenizer
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import nltk
import seaborn as sns ,sklearn,gensim
from nltk.corpus import stopwords
from collections import  Counter
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Read the data from https://github.com/adelmeleka/AQAD

In [2]:
with open('AQAD-master\AQQAD 1.0\FINAL_AAQAD-v1.0.json','rb') as f:
  data = json.load(f)

for allData in data['data']:
    print(allData['title']+"\n")
    print(allData['paragraphs'])
    break

بيونسيه

[{'qas': [{'question': 'متى بدأت بيونسي تصبح شعبية؟', 'id': 1, 'answers': [{'text': 'في أواخر التسعينات', 'answer_start': 220}], 'is_impossible': False}, {'question': 'ما هي المجالات التي تنافس عليها بيونسيه عندما كانت تكبر؟', 'id': 2, 'answers': [{'text': 'غناء ورقص', 'answer_start': 173}], 'is_impossible': False}, {'question': 'متى غادرت بيونسي طفل القدر وتصبح مغنية منفردة؟', 'id': 3, 'answers': [{'text': '(2003)،', 'answer_start': 515}], 'is_impossible': False}, {'question': 'في أي مدينة و ولاية نشأت بيونسيه؟', 'id': 4, 'answers': [{'text': 'هيوستن بولاية تكساس،', 'answer_start': 88}], 'is_impossible': False}, {'question': 'في أي عقد أصبحت بيونسي مشهورة؟', 'id': 5, 'answers': [{'text': 'أواخر التسعينات', 'answer_start': 223}], 'is_impossible': False}, {'question': 'في أي مجموعة R&B كانت هي المغنية الرئيسية؟', 'id': 6, 'answers': [{'text': 'دستنيز تشايلد', 'answer_start': 290}], 'is_impossible': False}, {'question': 'ما الألبوم الذي جعلها فنانة معروفة عالميا؟', 'id': 7, 'ans

# Preprocessing

In [3]:
def parse_data(path):
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    # initialize lists for contexts, questions, and answers
    contexts = []
    questions = []
    answers = []
    # iterate through all data in squad data
    counter = 0
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                if 'plausible_answers' in qa.keys():
                    access = 'plausible_answers'
                else:
                    access = 'answers'
                for answer in qa[access]:
                    counter+=1
                    # append data to lists
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    # return formatted data lists
    print(counter)
    return contexts, questions, answers

In [4]:
contexts, questions, answers = parse_data('AQAD-master\AQQAD 1.0\FINAL_AAQAD-v1.0.json')
#splitting the data into train and test
from sklearn.model_selection import train_test_split
train_contexts, val_contexts, train_questions, val_questions, train_answers, val_answers = train_test_split(contexts, questions, answers, test_size=0.2, random_state=42)

17896


In [5]:
print(len(train_questions))
display(train_questions[0:3])
print(len(train_answers))
display(train_answers[0:3])
print(len(train_contexts))
display(train_contexts[0:3])
print('-----------------------------------------')
print(len(val_questions))
display(val_questions[0:3])
print(len(val_answers))
display(val_answers[0:3])
print(len(val_contexts))
display(val_contexts[0:3])

14316


['ما هي أكبر القطاعات الاقتصادية في برمودا؟',
 'من الذي أرسل ليحل محل المواقف المطرودة؟',
 'كم سنة يمكن تخزين الذاكرة على المدى القصير؟']

14316


[{'text': 'التأمين وإعادة التأمين والسياحة،', 'answer_start': 25},
 {'text': 'الروس', 'answer_start': 192},
 {'text': 'من', 'answer_start': 24}]

14316


...

['ماذا يجب أن يحدث إذا ركزت فقط على إرضاء الذات؟',
 'ما هي الأعراض التي ستحدث إذا تسبب مرض السل في تمدد الأوعية الدموية في راسموسن؟',
 'ما ينمو على نقل الأنهار الجليدية في ألاسكا؟']

3580


[{'text': 'بيد أن القاعدة التي يتم خرقها ربما لا تكون', 'answer_start': 330},
 {'text': 'نزف', 'answer_start': 367},
 {'text': 'النباتات', 'answer_start': 73}]

3580


['إن فعل العصيان المدني هو خرق واع ومتعمد للقانون. كما أنه انتهاك لقاعدة قانونية وضعية. فإذا كان الانتهاك يقوم على القاعدة محل الخلاف مباشرة، فنحن بصدد الحديث عن عصيان مباشر. وهذا هو الحال، على سبيل المثال، بالنسبة لحملة العصيان المدني التي أطلقها مارتن لوثر كينغ والتي كانت تهدف إلى شغل السود للأماكن المخصصة بمقتضى للقانون للبيض. بيد أن القاعدة التي يتم خرقها ربما لا تكون هي تلك محل الخلاف، وحينئذ يكون الحديث عن عصيان مدني غير مباشر، وهو الحال- على سبيل المثال- بالنسبة للاعتصامات التي لا تهدف إلى الاحتجاج على قانون المرور.\nوعلى الرغم من عدم إمكانية إدراك وجود خرق مسبق (فالقاضي وحده هو الذي يحدد وجود خرق من عدمه)، فإن أي فعل يعد عصيانا مدنيا، عندما يخاطر القائمون عليه بعمل يكون، في نظر الرأي العام ورأي السلطات، خرقا عاما للقانون.\nوعند التعرض لهذه القضية، يجدر التذكير بالتجربة التي قام بها ستانلي ميلجرام. والتي تمثلت في قياس نسبة الأفراد القادرين على إطلاق مثل هذا العمل الخاص بالعصيان على الرغم من وجود ضغوط اجتماعية وإدارية.\n',
 'إذا تحولت عدوى السل إلى حالة نشطة، فإنها ستُصيب الرئتين

## Preprocessing the dataset using arabert in addition to this SQuAD preprocessing guide https://www.youtube.com/watch?v=ZIRmXkHp0-c&t=287s

In [6]:
def add_end_idx(answers, contexts):
    # loop through each answer-context pair
    for answer, context in zip(answers, contexts):
      # gold_text refers to the answer we are expecting to find in context
      gold_text = answer['text']
      # we already know the start index
      start_idx = answer['answer_start']
      # and ideally this would be the end index...
      end_idx = start_idx + len(gold_text)
      # ...however, sometimes squad answers are off by a character or two
      if context[start_idx:end_idx] == gold_text:
        # if the answer is not off :)
        answer['answer_end'] = end_idx
      else:
        for n in [1, 2]:
            if context[start_idx+n:end_idx+n] == gold_text:
                # this means the answer is off by 'n' tokens
                answer['answer_start'] = start_idx + n
                answer['answer_end'] = end_idx + n
                
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [7]:
#loop over answers, and contexts and make sure that they match
def check_answers_contexts_match_and_find_anomalies(answers, contexts, questions, show_anomalies=False):
    
    answers_without_answer_end_key =0
    non_matching_answers=0
    indices_of_answers_without_answer_end_key = []
    for answer, context, question in zip(answers, contexts, questions):
        
        #check if context has key answer_end and if not 
        if 'answer_end' not in answer.keys():
            answers_without_answer_end_key +=1
            if show_anomalies:
                print(question)
                print(answer)
                print(context[answer['answer_start']:answer['answer_start']+ len(answer['text'])+1])
            indices_of_answers_without_answer_end_key.append(answers.index(answer))

            continue 

        #check if answer matches context
        if context[answer['answer_start']:answer['answer_end']] != answer['text']:
            non_matching_answers +=1
    display('answers without answer_end key:', answers_without_answer_end_key,'non matching answers:', non_matching_answers)
    return indices_of_answers_without_answer_end_key


print("before dropping:")
print(len(train_questions))
print(len(train_answers))
print(len(train_contexts))

indices_to_drop1 = check_answers_contexts_match_and_find_anomalies(train_answers, train_contexts, train_questions, True)
#drop the answers, contexts and questions that do not match the context
train_answers = [i for j, i in enumerate(train_answers) if j not in indices_to_drop1]
train_contexts = [i for j, i in enumerate(train_contexts) if j not in indices_to_drop1]
train_questions = [i for j, i in enumerate(train_questions) if j not in indices_to_drop1]
indices_to_drop2 = check_answers_contexts_match_and_find_anomalies(val_answers, val_contexts, val_questions)
#drop the answers, contexts and questions that do not match the context
val_answers = [i for j, i in enumerate(val_answers) if j not in indices_to_drop2]
val_contexts = [i for j, i in enumerate(val_contexts) if j not in indices_to_drop2]
val_questions = [i for j, i in enumerate(val_questions) if j not in indices_to_drop2]
# print length of data after dropping

print("dropping",len(indices_to_drop1), "indices from train")

print(len(train_questions))
print(len(train_answers))
print(len(train_contexts))

print("dropping",len(indices_to_drop2), "indices from train")
print(len(val_questions))
print(len(val_answers))
print(len(val_contexts))


before dropping:
14316
14316
14316


'answers without answer_end key:'

0

'non matching answers:'

0

'answers without answer_end key:'

0

'non matching answers:'

0

dropping 0 indices from train
14316
14316
14316
dropping 0 indices from train
3580
3580
3580


dropping the triplets which don't have answer_end in their answers

## Encoding

In [8]:
# old
arabert_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabert",
                                                    do_lower_case=False,
                                                    do_basic_tokenize=True,
                                                    remove_html_markup = False,
                                                    strip_tashkeel = True,
                                                    strip_tatweel = True,
                                                    keep_emojis = True,
                                                    insert_white_spaces = True,
                                                    replace_slash_with_dash = True,
                                                    apply_farasa_segmentation = False)

# new 
# arabert_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabert",
#                                                     do_lower_case=True,
#                                                     do_basic_tokenize=True,
#                                                     remove_html_markup = False,
#                                                     strip_tashkeel = True,
#                                                     strip_tatweel = True,
#                                                     keep_emojis = True,
#                                                     insert_white_spaces = True,
#                                                     replace_slash_with_dash = True,
#                                                     apply_farasa_segmentation = False)

In [9]:
# tokenizer = arabert_tokenizer
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-multilingual-cased')
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)



In [10]:
def show_encoding_of(sentence, tokenizer):
    encoding = tokenizer(sentence)
    
    print('length of encoding:', len(encoding['input_ids']))
    print('number of words around splitted spaces :', len(sentence.split(' ')))
    list_of_tokens = []
    count_of_unk_tokens = 0
    for i in encoding['input_ids']:
        #print the decoding of it
        list_of_tokens.append(tokenizer.decode([i]))
        if(tokenizer.decode([i]) == '[UNK]'):
            count_of_unk_tokens +=1
    #print the list of tokens
    print('number of tokens that are not in the vocab:', count_of_unk_tokens)
    print(list_of_tokens[1:-1], "\n")
    # print(encoding)
    
# test_text = "الألواح الخشبية تعالج بعد ذلك في أفران للتحميص حتى لا تتأثر مستقبلاً بتغييرات درجات الحرارة أو الرطوبة الموجودة في الجو"
# test_text = "يوجد في باريس العديد من الملاعب المخصصة لمختلف أنواع الرياضات. يعدّ ملعب فرنسا الذي يتسع لأكثر من 80 ألف متفرج أكبر ملاعب البلاد، وكان قد بني هذا الملعب الواقع في منطقة سان دينس لاستضافة كأس العالم لكرة القدم 1998 والذي حازت عليه فرنسا للمرة الأولى في تاريخها. يستخدم الملعب لممارسة كرة القدم، والرجبي، وألعاب القوى. يستضيف الملعب مباريات منتخب فرنسا الوطني للرجبي سنوياً في بطولة الأمم الستة. وكذلك يستضيف مباريات منتخب فرنسا لكرة القدم الودية وتصفيات البطولات الكبرى. بالإضافة إلى نادي باريس سان جيرمان، تملك المدينة نوادي كرة قدم أخرى مثل: نادي باريس، والنجم الأحمر ونادي فرنسا"
for i in range(0,4):
    test_text = train_contexts[i]
    show_encoding_of(test_text, tokenizer)


length of encoding: 199
number of words around splitted spaces : 96
number of tokens that are not in the vocab: 0
['و', '##يس', '##تن', '##د', 'اقتصاد', 'بر', '##مو', '##دا', 'على', 'ال', '##ت', '##أمين', 'و', '##إ', '##عادة', 'ال', '##ت', '##أمين', 'و', '##ال', '##سي', '##احة', '،', 'وهم', '##ا', 'أكبر', 'ق', '##طاع', '##ين', 'اقتصاد', '##يين', 'في', 'الجزيرة', '.', 'وكانت', 'بر', '##مو', '##دا', 'صاحب', '##ة', 'إحدى', 'أعلى', 'معدل', '##ات', 'ال', '##نات', '##ج', 'ال', '##م', '##حلي', 'ال', '##إ', '##جمالي', 'ل', '##لف', '##رد', 'في', 'العالم', 'في', 'معظم', 'سنوات', 'القرن', 'العشرين', 'و', '##ل', '##عدة', 'سنوات', 'بعدها', '.', 'ت', '##أثر', 'وضع', '##ها', 'ال', '##اق', '##ت', '##صاد', '##ي', 'في', 'ال', '##آ', '##ونة', 'الأخيرة', 'بسبب', 'ال', '##ر', '##كو', '##د', 'العالمي', '.', 'من', '##اخ', 'الجزيرة', 'هو', 'شبه', 'است', '##وا', '##ئي', '.', 'وب', '##رم', '##ود', '##ا', 'هي', 'أ', '##قصى', 'نقطة', 'في', 'شمال', 'مثل', '##ث', 'بر', '##مو', '##دا', '،', 'وهي', 'منطقة', 'من', 'ال

In [11]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        # append start/end token position using char_to_token method
        
        if('answer_end' not in answers[i]):
            continue
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))
        

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift one token forward
        go_back = 1
        # print(end_positions[-1])
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end']-go_back)
            go_back +=1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

# apply function to our data
add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [12]:
print(dir(train_encodings))
display(train_encodings.data.keys())
print(len(train_encodings.data['input_ids']))
print(len(train_encodings.data['attention_mask']))
print(len(train_encodings.data['start_positions']))
print(len(train_encodings.data['end_positions']))
print(tokenizer.decode( train_encodings.data['input_ids'][0]))




['_MutableMapping__marker', '__abstractmethods__', '__class__', '__class_getitem__', '__contains__', '__copy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_encodings', '_n_sequences', 'char_to_token', 'char_to_word', 'clear', 'convert_to_tensors', 'copy', 'data', 'encodings', 'fromkeys', 'get', 'is_fast', 'items', 'keys', 'n_sequences', 'pop', 'popitem', 'sequence_ids', 'setdefault', 'to', 'token_to_chars', 'token_to_sequence', 'token_to_word', 'tokens', 'update', 'values', 'word_ids', 'word_to_chars', 'word_to_toke

dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])

14316
14316
14316
14316
[CLS] ويستند اقتصاد برمودا على التأمين وإعادة التأمين والسياحة ، وهما أكبر قطاعين اقتصاديين في الجزيرة. وكانت برمودا صاحبة إحدى أعلى معدلات الناتج المحلي الإجمالي للفرد في العالم في معظم سنوات القرن العشرين ولعدة سنوات بعدها. تأثر وضعها الاقتصادي في الآونة الأخيرة بسبب الركود العالمي. مناخ الجزيرة هو شبه استوائي. وبرمودا هي أقصى نقطة في شمال مثلث برمودا ، وهي منطقة من البحر اختفت فيها عدة سفن وطائرات في ظروف غامضة كما تقول الأسطورة. تقع الجزيرة في حزام الأعاصير وهي عرضة للرياح العاتية. ومع ذلك ، فهي محمية نوعا ما من كامل القوة التدميرية للأعاصير بفضل الشعاب المرجانية المحيطة بها. [SEP] ما هي أكبر القطاعات الاقتصادية في برمودا ؟ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

# Importing the model

In [13]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
      return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)
    def convert_idx_to_text(input_ids, idx):
        return tokenizer.decode(input_ids[idx], skip_special_tokens=True)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)
from transformers import DistilBertForQuestionAnswering
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-multilingual-cased")

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should pr

In [14]:
print(len(train_dataset))

14316


# Finetuning

In [15]:
!nvidia-smi

Wed May 17 22:30:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   68C    P0    23W /  N/A |    465MiB /  6144MiB |      7%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [52]:
import torch

# set the max_split_size_mb parameter to 1024 MB
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cuda.max_split_size_mb = 1024

torch.cuda.empty_cache()
print("memory_allocated",torch.cuda.memory_allocated())
print("max_memory_allocated",torch.cuda.max_memory_allocated())

memory_allocated 5098476032
max_memory_allocated 5098476032


In [51]:
from torch.utils.data import DataLoader
from transformers import AdamW

from tqdm import tqdm
import torch
import random
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move model over to detected device
model.to(device)
# activate training mode of model
model.train()
# initialize adam optimizer with weight decay (reduces chance of overfitting)
optim = AdamW(model.parameters(), lr=5e-5)
# try increase batch size to 16 on colab 
# initialize data loader for training data
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, worker_init_fn=np.random.seed(seed))
for epoch in range(6):
    # set model to train mode
    model.train()
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, leave=True)
    total_loss = 0.0
    num_batches = 0
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                      end_positions=end_positions)
        loss = outputs[0]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        total_loss += loss.item()
        num_batches += 1
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item(), avg_loss=total_loss / num_batches)
        avg_epoch_loss = total_loss / num_batches
    
    #evaluate the model after the epoch
    # switch model out of training mode
    model.eval()
    #val_sampler = SequentialSampler(val_dataset)
    val_loader = DataLoader(val_dataset, batch_size=2,  shuffle=True, worker_init_fn=np.random.seed(seed))
    acc = []
    # initialize loop for progress bar
    loop = tqdm(val_loader, leave=True)
    # loop through batches
    for batch in loop:
        # we don't need to calculate gradients as we're not training
        with torch.no_grad():
            # pull batched items from loader
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_true = batch['start_positions'].to(device)
            end_true = batch['end_positions'].to(device)
            # make predictions
            outputs = model(input_ids, attention_mask=attention_mask)
            # pull preds out
            start_pred = torch.argmax(outputs['start_logits'], dim=1)
            end_pred = torch.argmax(outputs['end_logits'], dim=1)
            # calculate accuracy for both and append to accuracy list
            acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
            acc.append(((end_pred == end_true).sum()/len(end_pred)).item())
            loop.set_description(f'Epoch {epoch}')
            loop.set_postfix(avg_acc=sum(acc)/len(acc))
            # avg_acc = sum(acc)/len(acc)


  0%|          | 0/895 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 6.00 GiB total capacity; 4.75 GiB already allocated; 0 bytes free; 4.75 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Saving the model

In [ ]:
def getNewPathName(model_path):
  if os.path.exists(model_path):
    model_name = os.path.basename(model_path)  # Get the base name of the model path
    base_path = os.path.dirname(model_path)  # Get the directory path

    # Find the current number at the end of the model name (e.g., "distilbert-all1")
    model_number = 1
    while os.path.exists(model_path):
        model_number += 1
        model_path = os.path.join(base_path, f"{model_name}{model_number}")
    return model_path
model_path = getNewPathName('/content/drive/MyDrive/GUC related/NLP_Project_data/models/distilbert-all1')
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/GUC related/NLP_Project_data/models/distilbert-all13/tokenizer_config.json',
 '/content/drive/MyDrive/GUC related/NLP_Project_data/models/distilbert-all13/special_tokens_map.json',
 '/content/drive/MyDrive/GUC related/NLP_Project_data/models/distilbert-all13/vocab.txt',
 '/content/drive/MyDrive/GUC related/NLP_Project_data/models/distilbert-all13/added_tokens.json',
 '/content/drive/MyDrive/GUC related/NLP_Project_data/models/distilbert-all13/tokenizer.json')

# Evaluation

In [ ]:
# switch model out of training mode
model.eval()

#val_sampler = SequentialSampler(val_dataset)
val_loader = DataLoader(val_dataset, batch_size=16,  shuffle=True, worker_init_fn=np.random.seed(seed))

acc = []

# initialize loop for progress bar
loop = tqdm(val_loader)
# loop through batches
for batch in loop:
    # we don't need to calculate gradients as we're not training
    with torch.no_grad():
        # pull batched items from loader
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_true = batch['start_positions'].to(device)
        end_true = batch['end_positions'].to(device)
        # make predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        # pull preds out
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
        # calculate accuracy for both and append to accuracy list
        acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
        acc.append(((end_pred == end_true).sum()/len(end_pred)).item())
# calculate average accuracy in total


100%|██████████| 224/224 [00:58<00:00,  3.86it/s]


In [ ]:
avg_acc = sum(acc)/len(acc)
print(acc)
print(avg_acc)

[0.25, 0.125, 0.125, 0.125, 0.125, 0.1875, 0.1875, 0.125, 0.3125, 0.375, 0.1875, 0.1875, 0.3125, 0.125, 0.25, 0.125, 0.25, 0.25, 0.3125, 0.25, 0.1875, 0.125, 0.1875, 0.3125, 0.3125, 0.375, 0.1875, 0.1875, 0.125, 0.1875, 0.1875, 0.25, 0.25, 0.3125, 0.3125, 0.375, 0.125, 0.125, 0.1875, 0.125, 0.1875, 0.1875, 0.0, 0.125, 0.1875, 0.1875, 0.125, 0.1875, 0.25, 0.5, 0.0625, 0.0625, 0.1875, 0.3125, 0.25, 0.1875, 0.0625, 0.125, 0.125, 0.1875, 0.25, 0.1875, 0.5, 0.375, 0.3125, 0.375, 0.125, 0.0625, 0.25, 0.25, 0.0625, 0.3125, 0.1875, 0.0625, 0.1875, 0.1875, 0.25, 0.25, 0.125, 0.125, 0.1875, 0.1875, 0.25, 0.375, 0.125, 0.125, 0.1875, 0.25, 0.125, 0.125, 0.3125, 0.25, 0.0625, 0.1875, 0.25, 0.375, 0.375, 0.375, 0.3125, 0.25, 0.3125, 0.3125, 0.125, 0.0, 0.0625, 0.1875, 0.125, 0.1875, 0.25, 0.1875, 0.125, 0.0625, 0.25, 0.25, 0.1875, 0.3125, 0.125, 0.125, 0.1875, 0.1875, 0.25, 0.1875, 0.1875, 0.25, 0.5625, 0.5, 0.125, 0.125, 0.375, 0.125, 0.25, 0.3125, 0.25, 0.3125, 0.25, 0.1875, 0.375, 0.4375, 0.125,

In [ ]:
import Levenshtein

# ...

tolerance = 0.9  # Set your desired tolerance level between 0 and 1

exact_match_count = 0
total_count = 0

# loop through batches
for batch in loop:
    # ...

    # pull preds out
    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)

    # convert indices to strings
    start_pred_str = [val_dataset.convert_idx_to_text(input_ids[i], start_pred[i].item()) for i in range(len(start_pred))]
    end_pred_str = [val_dataset.convert_idx_to_text(input_ids[i], end_pred[i].item()) for i in range(len(end_pred))]

    start_true_str = [val_dataset.convert_idx_to_text(input_ids[i], start_true[i].item()) for i in range(len(start_true))]
    end_true_str = [val_dataset.convert_idx_to_text(input_ids[i], end_true[i].item()) for i in range(len(end_true))]

    # calculate exact match with tolerance for both start and end positions
    for pred, true in zip(start_pred_str, start_true_str):
        if Levenshtein.ratio(pred, true) >= tolerance:
            exact_match_count += 1
        total_count += 1

    for pred, true in zip(end_pred_str, end_true_str):
        if Levenshtein.ratio(pred, true) >= tolerance:
            exact_match_count += 1
        total_count += 1

# calculate exact match with tolerance accuracy
exact_match_tolerance_accuracy = exact_match_count / total_count if total_count != 0 else 0


TypeError: ignored

In [ ]:
print("T/F\tstart\tend\n")
for i in range(len(start_true)):
    print(f"true\t{start_true[i]}\t{end_true[i]}\n"
          f"pred\t{start_pred[i]}\t{end_pred[i]}\n")